Extracting audios

In [ ]:
import os
from moviepy.editor import VideoFileClip

def extract_audio(video_path, audio_output_path):
    try:
        video_clip = VideoFileClip(video_path)
        audio_clip = video_clip.audio
        audio_clip.write_audiofile(audio_output_path)
        print(f"Audio extracted successfully from {video_path}!")
    except Exception as e:
        print(f"Error extracting audio from {video_path}: {e}")

def extract_audio_from_all_videos(video_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file in os.listdir(video_folder):
        if file.endswith(".mp4"):
            video_file_path = os.path.join(video_folder, file)
            audio_output_path = os.path.join(output_folder, os.path.splitext(file)[0] + ".wav")
            extract_audio(video_file_path, audio_output_path)

# Example usage:
video_folder = r".."  # Provide the path to the folder containing video files
output_folder = "Audios"  # Specify the folder where you want to save the extracted audio files
extract_audio_from_all_videos(video_folder, output_folder)


Audio Features Analysis

In [4]:
from pathlib import Path
import time
import parselmouth as pm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

audio_Column = ['Video Name', 'Length', 'Average Band Energy', 'Avg Intensity', 'Max Intensity', 'Mean Intensity', 'Range Intensity', 'SD Intensity', 
		   'Avg Pitch', 'Max Pitch', 'Mean Pitch', 'Range Pitch', 'SD Pitch',
		   'Mean F1', 'Mean F2', 'Mean F3', 'Mean B1', 'Mean B2', 'Mean B3', 'SD F1', 'SD F2', 'SD F3', 
		   'Mean F2/F1', 'Mean F3/F1', 'SD F2/F1', 'SD F3/F1']
audio_row = []

def amplitude(sound):
	x_sample = sound.xs()
	amplitude = sound.values[:,0]

	# print('Time: ',x_sample)
	# print('Amplitude: ',amplitude)
	
	# plt.figure()
	# plt.plot(x_sample, amplitude)
	# plt.xlim([sound.xmin, sound.xmax])
	# plt.xlabel("Time [s]")
	# plt.ylabel("Amplitude")

def intensity(sound):
	intensity = sound.to_intensity()
	
	x_sample = intensity.xs()
	y_intensity = intensity.values
	# print('Time: ',x_sample)
	# print('Intensity: ',y_intensity)

	avg_intensity = intensity.get_average(intensity.end_time,intensity.start_time,'ENERGY')
	max_intensity = np.max(y_intensity)
	min_intensity = np.min(y_intensity)
	range_intensity = max_intensity - min_intensity
	sd_intensity = np.std(y_intensity)
	# print(sd_intensity)
	
	# plt.figure()
	# plt.plot(x_sample, y_intensity, linewidth=3, color='w')
	# plt.plot(x_sample, y_intensity, linewidth=1)
	# plt.grid(False)
	# plt.xlim(intensity.start_time,intensity.end_time)
	# plt.xlabel("Time [s]")
	# plt.ylabel("intensity [dB]")
	audio_row.extend([avg_intensity, max_intensity, min_intensity, range_intensity, sd_intensity])

def pitch(sound):
	pitch = sound.to_pitch()
	
	x_sample = pitch.xs()
	y_pitch = pitch.to_matrix().values
	# print('Time: ',x_sample)
	# print('Pitch: ',y_pitch)
	
	y_pitch[y_pitch == 0] = np.nan

	avg_pitch =	np.nanmean(y_pitch)
	max_pitch =	np.nanmax(y_pitch)
	min_pitch =	np.nanmin(y_pitch)
	range_pitch	= max_pitch - min_pitch
	sd_pitch = np.nanstd(y_pitch)
	# print(sd_pitch)

	# plt.figure()
	# plt.plot(x_sample, y_pitch, linewidth=3, color='w')
	# plt.plot(x_sample, y_pitc, linewidth=1)
	# plt.grid(False)
	# plt.xlim(intensity.start_time,intensity.end_time)
	# plt.xlabel("Time [s]")
	# plt.ylabel("Frequency [dB]")
	audio_row.extend([avg_pitch, max_pitch, min_pitch, range_pitch, sd_pitch])


def formant(sound):
	formant = sound.to_formant_burg(max_number_of_formants = 5)
	f1 = []
	b1 = []
	f2 = []
	b2 = []
	f3 = []
	b3 = []

	x_sample = formant.xs()
	# print(x_sample)
	
	# start_time_formant = formant.get_start_time()
	# end_time_formant = formant.get_end_time()
	# time_step_formant = formant.get_time_step()
	# print(len(np.linspace(0.027,end_time_formant,len(formant.xs()))))
	# print(len(np.arange(0.027,end_time_formant,time_step_formant)))
	# print(start_time_formant,end_time_formant,time_step_formant)
	# print('Time: ',x_sample)
	# print(formant.get_value_at_time(1,0.039513))
	# print(formant.get_value_at_time(1,0.03325792))
	# print('Pitch: ',y_formant)

	for x in x_sample:
		f1.append(formant.get_value_at_time(1,x))
		f2.append(formant.get_value_at_time(2,x))
		f3.append(formant.get_value_at_time(3,x))
		b1.append(formant.get_bandwidth_at_time(1,x))
		b2.append(formant.get_bandwidth_at_time(2,x))
		b3.append(formant.get_bandwidth_at_time(3,x))

	mean_f1 = np.mean(f1)
	mean_f2 = np.mean(f2)
	mean_f3 = np.mean(f3)

	mean_b1 = np.mean(b1)
	mean_b2 = np.mean(b2)
	mean_b3 = np.mean(b3)

	sd_f1 = np.std(f1)
	sd_f2 = np.std(f2)
	sd_f3 = np.std(f3)

	mean_f2_by_f1 = np.mean(np.array(f2)/np.array(f1))
	mean_f3_by_f1 = np.mean(np.array(f3)/np.array(f1))
	
	sd_f2_by_f1 = np.std(np.array(f2)/np.array(f1))
	sd_f3_by_f1 = np.std(np.array(f3)/np.array(f1))

	audio_row.extend([mean_f1, mean_f2, mean_f3, mean_b1, mean_b2, mean_b3, sd_f1, sd_f2, sd_f3, 
				mean_f2_by_f1, mean_f3_by_f1, sd_f2_by_f1, sd_f3_by_f1])


def spectrum(sound):
	spectrum = sound.to_spectrum()
	band_energy_spectrum = spectrum.get_band_energy()
	# print(band_energy_spectrum)
	# spectrogram = spectrum.to_spectrogram()
	# drawSpectrogram(sound)
	audio_row.append(band_energy_spectrum)

def drawSpectrogram(sound, dynamic_range=70):

	spectrogram = sound.to_spectrogram(window_length=0.05)
	X, Y = spectrogram.x_grid(), spectrogram.y_grid()
	sg_db = 10 * np.log10(spectrogram.values.T)
	plt.pcolormesh(X, Y, sg_db, vmin=sg_db.max() - dynamic_range, cmap='afmhot')
	plt.ylim([spectrogram.ymin, spectrogram.ymax])
	plt.xlabel("time [s]")
	plt.ylabel("frequency [Hz]")
	
	plt.xlim([sound.xmin, sound.xmax])




def audio_analysis():
    
    
    ## write our code for video taking
    print("starting Audio analysis")
    global audio_row
    data = []
    current_directory = os.getcwd()
    audios_directory = os.path.join(current_directory, "Audios")  # Construct the path as a string
    print(audios_directory)
    for file in os.listdir(audios_directory):
        filename = os.path.join(audios_directory, file)
        
        if filename.endswith(".wav") or filename.endswith(".mp3"): 
            sound = pm.Sound(filename)
            print(filename)
            audio_row.append(filename)
            end_time = sound.get_total_duration()
            audio_row.append(end_time)
            amplitude(sound)
            spectrum(sound)
            intensity(sound)
            pitch(sound)
            formant(sound)
            data.append(audio_row)
            print(audio_row)
            audio_row = []
            # index +=1
            
        
        df = pd.DataFrame(data = data, columns = audio_Column)
        df.to_csv('audioCues.csv')
        # plt.show()
        
    else:
        print("Some Process in running file")

In [ ]:
%pip install moviepy

In [ ]:
%pip install praat-parselmouth


In [ ]:
audio_analysis()

In [ ]:
import pandas as pd

file_path = "audio_labels.csv"

df = pd.read_csv(file_path)
print(df.head())


In [ ]:
aggr_worker_rows = df[df['Worker'] == 'AGGR'] # aggregate of workers' prediction
selected_columns = ['Participant', 'Focused', 'EngagingTone', 'Excited', 'SpeakingRate', 'Calm', 'StructuredAnswers', 'Paused', 'NoFillers', 'Friendly']
aggr_worker_data = aggr_worker_rows.loc[:, selected_columns]
print(aggr_worker_data)

In [18]:
aggr_worker_data.to_csv('labels.csv', index=False)

In [26]:
import pandas as pd

# Load the input and output data
input_data = pd.read_csv('audioCues.csv')
output_data = pd.read_csv('labels.csv')

# Extract the participant names from the 'Video Name' column in the input data
input_data['Participant'] = input_data['Video Name'].apply(lambda x: x.split('\\')[-1].split('.')[0].lower())

# Convert 'Participant' column in output data to lowercase
output_data['Participant'] = output_data['Participant'].str.lower()

# Extract the rows from the output data where the participant matches the participant names extracted from the input data
matched_output_data = output_data[output_data['Participant'].isin(input_data['Participant'])]

# Print the matched output data
print(matched_output_data)

matched_output_data.to_csv('matched_labels.csv', index=False)


    Participant   Focused  EngagingTone   Excited  SpeakingRate      Calm  \
0            p1  5.845226      5.147909  5.043890      4.865590  5.351075   
1            p3  5.549829      5.621231  5.601586      5.016286  4.476537   
2            p4  5.324136      4.392736  4.261343      4.738000  5.065190   
3            p5  6.322086      4.695523  4.707062      4.927181  6.130488   
4            p6  5.619420      4.261988  4.648259      4.174891  5.828977   
5            p7  6.197476      4.941384  4.886050      4.914479  6.007913   
6            p8  5.823738      4.157069  4.233973      4.453324  5.662728   
7           p10  5.267886      4.819107  4.383947      4.775369  4.445185   
8           p11  5.717776      4.426753  4.297760      4.895461  5.029086   
9           p12  5.464584      5.452386  5.322526      5.246078  5.202629   
46          p62  5.144276      4.736510  4.666732      4.368588  5.281840   
47          p63  5.461772      4.311239  3.909425      4.053482  5.493760   

In [36]:
import pandas as pd

# Read participant data from the CSV file
participant_data = pd.read_csv('matched_labels.csv')

# Convert participant data into vectors
participant_vectors = {}
for index, row in participant_data.iterrows():
    participant = row['Participant']
    vector = row.drop('Participant').values
    participant_vectors[participant] = vector

# Display participant vectors
for participant, vector in participant_vectors.items():
    print(f"Participant: {participant}, Vector: {vector}")


Participant: p1, Vector: [5.84522637357 5.14790937729 5.04389015411 4.86558979541 5.35107543307
 4.89158004615 5.80046790524 3.77166473066 5.25478355985]
Participant: p3, Vector: [5.54982872237 5.62123092399 5.60158567429 5.01628608696 4.47653726578
 3.9121988684 4.79154954828 3.33245849049 6.50032877227]
Participant: p4, Vector: [5.3241358317 4.39273648876 4.26134323401 4.73800014854 5.06519019504
 4.68837924196 5.17744701629 3.11144466161 5.36988386167]
Participant: p5, Vector: [6.32208591415 4.69552295169 4.70706218787 4.92718068841 6.13048798847
 5.58251442294 5.88632626708 5.88174118567 5.64911869128]
Participant: p6, Vector: [5.61942047508 4.26198846721 4.64825935206 4.17489068485 5.8289772252
 4.82624462496 5.02782250409 3.92032941437 4.49537499283]
Participant: p7, Vector: [6.19747572924 4.94138360782 4.88604973191 4.91447854325 6.00791295588
 5.57242669885 6.04631607177 5.92916127869 5.50456862011]
Participant: p8, Vector: [5.8237382037 4.15706890643 4.23397310045 4.4533243525

In [38]:
import pandas as pd

# Read audio cues data from the CSV file
audio_data = pd.read_csv('audioCues.csv')


# Extract participant from 'Video Name' (filename) and remove '.wav' to lowercase
audio_data['Participant'] = audio_data['Video Name'].apply(lambda x: x.split('\\')[-1].replace('.wav', '').lower())

# Remove index and 'Video Name' column
audio_data = audio_data.drop(columns=['Video Name', audio_data.columns[0]])

# Display the modified DataFrame
print(audio_data)


        Length  Average Band Energy  Avg Intensity  Max Intensity  \
0   240.000000             0.029007      54.842035      72.249658   
1   298.500000             0.044217      55.724548      69.922859   
2   285.800000             0.029588      54.169924      70.591093   
3   278.529977             0.008429      48.832036      64.578172   
4   261.300000             0.036603      55.484288      72.132899   
5   292.100000             0.066531      57.593880      83.230547   
6   267.970000             0.014751      51.427915      67.739448   
7   129.669977             0.010690      53.183336      66.742637   
8   187.030000             0.009693      51.167721      72.020565   
9   128.370000             0.014486      54.546701      71.640220   
10  187.300000             0.013081      52.461856      68.136799   
11  150.600000             0.008666      51.621258      64.688707   
12  189.770000             0.013681      52.608352      68.453386   
13  211.530000             0.05193

In [41]:
audio_data.to_csv('input.csv', index=False)

In [42]:
import pandas as pd

# Load input.csv and matched_labels.csv
input_df = pd.read_csv('input.csv')
labels_df = pd.read_csv('matched_labels.csv')

# Merge the DataFrames based on the 'Participant' column
combined_df = pd.merge(input_df, labels_df, on='Participant')

# Save the combined DataFrame to train.csv
combined_df.to_csv('train.csv', index=False)

print("Data saved to train.csv")


Data saved to train.csv


MODEL TRAINING CODE

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from joblib import dump

# Load the data
data = pd.read_csv('train-copy.csv')

# Separate features (input) and target (output)
X = data.drop(columns=['Participant', 'Focused', 'EngagingTone', 'Excited', 'SpeakingRate', 'Calm', 'StructuredAnswers', 'Paused', 'NoFillers', 'Friendly'])
y = data[['Focused', 'EngagingTone', 'Excited', 'SpeakingRate', 'Calm', 'StructuredAnswers', 'Paused', 'NoFillers', 'Friendly']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

dump(scaler, "scaler.joblib")

# Train individual SVR models for each target variable
models = {}
y_preds = {}
mses = {}

for column in y_train.columns:
    model = SVR(kernel='rbf')
    model.fit(X_train_scaled, y_train[column])
    models[column] = model
    
    dump(model, f"{column}_model.joblib")
    
    y_pred = model.predict(X_test_scaled)
    y_preds[column] = y_pred
    
    mse = mean_squared_error(y_test[column], y_pred)
    mses[column] = mse
    print(f"Mean Squared Error for {column}: {mse}")

# Overall Mean Squared Error
overall_mse = sum(mses.values()) / len(mses)
print("Overall Mean Squared Error:", overall_mse)


Mean Squared Error for Focused: 0.15720555949651416
Mean Squared Error for EngagingTone: 0.27350985969843616
Mean Squared Error for Excited: 0.35151985441864814
Mean Squared Error for SpeakingRate: 0.18090762982368283
Mean Squared Error for Calm: 0.44087849573267646
Mean Squared Error for StructuredAnswers: 0.35102196589476625
Mean Squared Error for Paused: 0.42073709567484613
Mean Squared Error for NoFillers: 0.43710781974991325
Mean Squared Error for Friendly: 0.2467095379842043
Overall Mean Squared Error: 0.31773309094152086


To process a single video file

In [5]:
import parselmouth as pm

def audio_file_analysis(audio_file):
    print("Starting Audio analysis")
    global audio_row 
    audio_row = []
    data = []
    try:
        sound = pm.Sound(audio_file)
        print(f"Analyzing audio file: {audio_file}")
        #audio_row.append(audio_file)
        end_time = sound.get_total_duration()
        audio_row.append(end_time)
        amplitude(sound)
        spectrum(sound)
        intensity(sound)
        pitch(sound)
        formant(sound)
        audio_row = [0 if pd.isna(value) else value for value in audio_row]
        data.append(audio_row)
        print(audio_row)
        return audio_row
    except Exception as e:
        print(f"Error analyzing audio file: {audio_file}")
        print(e)

# Example usage:
audio_file_path = "E:/Videos/EditedVideos/VHA-Interview/Audios/P63.wav"  # Replace with your audio file path
new_input = audio_file_analysis(audio_file_path)

Starting Audio analysis
Analyzing audio file: E:/Videos/EditedVideos/VHA-Interview/Audios/P63.wav
[278.52997732426303, 0.008428870744538393, 48.83203632863392, 64.57817206814136, 31.89362548221409, 32.684546585927265, 5.395295316784023, 130.39140838092655, 597.712617595605, 74.97801700759291, 522.7346005880121, 115.8888889905572, 721.951356881005, 1846.9013913478852, 0, 580.0397766156003, 567.0755284519188, 0, 391.8674838509295, 425.8443563124456, 0, 3.144205317140231, 0, 1.6729024612250905, 0]


In [6]:


from joblib import load

def loadVideoModels():
    # Dictionary to store loaded SVR models
    models = {}

    # List of target variable names
    target_variables = ['Focused', 'EngagingTone', 'Excited', 'SpeakingRate', 'Calm', 'StructuredAnswers', 'Paused', 'NoFillers', 'Friendly']

    # Load the saved SVR models
    for column in target_variables:
        model = load(f"{column}_model.joblib")
        models[column] = model
        
    return models


In [ ]:
# # # Assuming 'models' contains the trained SVR models for each target variable

# # Make sure 'new_input' is scaled using the same scaler used for training the models
# new_input_scaled = scaler.transform([new_input])  # Assuming 'new_input' is a list of features

# # Dictionary to store predicted values for each target variable
# predicted_values = {}

# # Iterate over each target variable and predict its value using the corresponding SVR model
# for column, model in models.items():
#     # Predict the value for the current target variable
#     predicted_value = model.predict(new_input_scaled)
#     predicted_values[column] = predicted_value

# # Print the predicted values
# for column, value in predicted_values.items():
#     print(f"Predicted {column}: {value}")


In [7]:
# Assuming 'models' contains the trained SVR models for each target variable

def predictVideaTraits(audio_file_path):
    models = loadVideoModels()
    scaler = load("scaler.joblib")
    new_input = audio_file_analysis(audio_file_path)
    new_input_scaled = scaler.transform([new_input])
# Make sure 'new_input' is scaled using the same scaler used for training the models
    new_input_scaled = scaler.transform([new_input])  # Assuming 'new_input' is a list of features

    # Dictionary to store predicted values for each target variable
    predicted_values = {}
    
    # Iterate over each target variable and predict its value using the corresponding SVR model
    for column, model in models.items():
        # Predict the value for the current target variable
        predicted_value = model.predict(new_input_scaled)
        # Scale the predicted value
        scaled_value = scale_prediction(predicted_value)
        predicted_values[column] = scaled_value

    # Print the predicted values
    for column, value in predicted_values.items():
        print(f"Predicted {column}: {value}")

# Define the scaling function
def scale_prediction(prediction):
    # Scale from 1-7 to 1-5
    return np.round(((prediction - 1) * (5 - 1) / (7 - 1)) + 1, 2)

predictVideaTraits(audio_file_path)


Starting Audio analysis
Analyzing audio file: E:/Videos/EditedVideos/VHA-Interview/Audios/P63.wav
[278.52997732426303, 0.008428870744538393, 48.83203632863392, 64.57817206814136, 31.89362548221409, 32.684546585927265, 5.395295316784023, 130.39140838092655, 597.712617595605, 74.97801700759291, 522.7346005880121, 115.8888889905572, 721.951356881005, 1846.9013913478852, 0, 580.0397766156003, 567.0755284519188, 0, 391.8674838509295, 425.8443563124456, 0, 3.144205317140231, 0, 1.6729024612250905, 0]
Predicted Focused: [4.09]
Predicted EngagingTone: [3.53]
Predicted Excited: [3.55]
Predicted SpeakingRate: [3.51]
Predicted Calm: [3.88]
Predicted StructuredAnswers: [3.52]
Predicted Paused: [4.04]
Predicted NoFillers: [3.21]
Predicted Friendly: [3.92]


c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
